In [1]:
# Notebook: 11_Pit_Stop_Power_Analysis.ipynb

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window

# --- 1. Configure and Start Spark Session ---
spark = SparkSession.builder \
    .appName("F1 Pit Stop Power Analysis") \
    .master("spark://spark-master:7077") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .getOrCreate()

print("Spark session created!")

# --- 2. Load All 24 Races ---
laps_dir = "s3a://raw-data/laps/"
print(f"Loading all lap data from {laps_dir}...")
df = spark.read.parquet(laps_dir)

# --- 3. Clean and Prepare Data ---
# We need to know which race each lap is from.
# We can get this from the file name.
df_with_race = df.withColumn("RaceName", F.regexp_extract(F.input_file_name(), r"(\w+)\.parquet$", 1))

# Filter for "good" laps
clean_df = df_with_race.filter(
    (F.col('IsAccurate') == True) &
    (F.col('LapTime').isNotNull()) &
    (F.col('Stint').isNotNull()) &
    (F.col('LapNumber').isNotNull()) &  # <-- Added this, it's needed for the Window
    (F.col('Driver').isNotNull()) &     # <-- Added this, it's needed for the Window
    (F.col('RaceName').isNotNull()) &   # <-- Added this, it's needed for the Window
    (F.col('Compound').isNotNull()) &
    (F.col('Compound') != 'UNKNOWN')
)

print(f"Total clean laps loaded: {clean_df.count()}")

# --- 4. Use Window Function to Find Pit Laps ---
print("Analyzing all laps to find pit stops...")

# We define a "Window" or "Context" for Spark.
# This tells Spark to group data by Race and Driver, and order it by Lap.
lap_window = Window.partitionBy("RaceName", "Driver").orderBy("LapNumber")

# Now we "slide" this window over the data.
# We ask Spark to get the value from the *previous* row ("lag")
# for 'Stint' and 'LapTime'.
laps_with_history_df = clean_df.withColumn("Prev_Stint", F.lag("Stint").over(lap_window)) \
                               .withColumn("Prev_LapTime", F.lag("LapTime").over(lap_window))

# --- 5. Isolate the "Post-Pit" Laps ---
# A "post-pit lap" is the first lap of a new stint.
# We find it by comparing the 'Stint' to the 'Prev_Stint'.
post_pit_laps_df = laps_with_history_df.filter(
    (F.col("Stint") != F.col("Prev_Stint")) &
    (F.col("Prev_Stint").isNotNull())
)

print(f"Found {post_pit_laps_df.count()} post-pit laps (new tire laps) across the season.")

# --- 6. Calculate the "Pit Stop Power" Delta ---
# 'LapTime' is the new, fast lap on fresh tires.
# 'Prev_LapTime' is the old, slow lap on worn tires.
# Delta = Old Lap - New Lap (A positive number means we got faster)
delta_df = post_pit_laps_df.withColumn(
    "Time_Delta", 
    F.col("Prev_LapTime") - F.col("LapTime")
)

# Show some examples
print("Sample of pit stop deltas (positive = faster):")
delta_df.select("RaceName", "Driver", "Compound", "LapTime", "Prev_LapTime", "Time_Delta").show(10)

# --- 7. Aggregate the Final Result ---
print("Calculating average time gained per compound...")
final_analysis = delta_df.groupBy("Compound") \
    .agg(
        F.avg("Time_Delta").alias("Average_Time_Gained"),
        F.count("Time_Delta").alias("Total_PitStops")
    ) \
    .orderBy("Average_Time_Gained", ascending=False)

# --- 8. Show and Save the Final "Product" ---
print("\n--- FINAL ANALYSIS: Pit Stop Power (Seconds) ---")
final_analysis.show()

# Save this final analysis to your processed data bucket
output_path = "s3a://processed-data/final_analysis/pit_stop_power.parquet"
print(f"Saving final analysis to {output_path}...")
final_analysis.write.mode("overwrite").parquet(output_path)

print("--- Analysis Complete and Saved! ---")
spark.stop()

Spark session created!
Loading all lap data from s3a://raw-data/laps/...
Total clean laps loaded: 23976
Analyzing all laps to find pit stops...
Found 716 post-pit laps (new tire laps) across the season.
Sample of pit stop deltas (positive = faster):
+---------------+------+--------+-------+------------+------------------+
|       RaceName|Driver|Compound|LapTime|Prev_LapTime|        Time_Delta|
+---------------+------+--------+-------+------------+------------------+
|2024_01_Bahrain|   ALB|    HARD| 97.392|      98.977| 1.585000000000008|
|2024_01_Bahrain|   ALB|    HARD| 96.129|      97.767| 1.637999999999991|
|2024_01_Bahrain|   ALO|    HARD|  96.97|      98.612| 1.641999999999996|
|2024_01_Bahrain|   ALO|    HARD| 95.107|      98.061|2.9540000000000077|
|2024_01_Bahrain|   BOT|    HARD| 96.454|      99.187| 2.733000000000004|
|2024_01_Bahrain|   BOT|    HARD| 96.763|      98.654|1.8909999999999911|
|2024_01_Bahrain|   GAS|    HARD| 97.689|      98.656| 0.967000000000013|
|2024_01_B